In [1]:
import sys, os
import argparse
import time
from datetime import datetime as dt
import gc; gc.enable()
from functools import partial, wraps

import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np # linear algebra
np.warnings.filterwarnings('ignore')

from sklearn.model_selection import StratifiedKFold
from tsfresh.feature_extraction import extract_features
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

from numba import jit

In [ ]:
z = pd.read_csv('subm_0.658553_2018-12-09-21-42.csv')

print(z.groupby('object_id').size().max())
print((z.groupby('object_id').size() > 1).sum())

z = z.groupby('object_id').mean()

z_new = z
z_new["sum"] = z_new.sum(axis=1)

z_new = z_new.loc[:,"class_6":"class_99"].div(z_new["sum"], axis=0)
z_new.index = z_new.index.map(int)

z2 = pd.read_csv('predictions.csv')

print(z2.groupby('object_id').size().max())
print((z2.groupby('object_id').size() > 1).sum())

z2 = z2.groupby('object_id').mean()

z2_new = z2
z2_new["sum"] = z2_new.sum(axis=1)

z2_new = z2_new.loc[:,"class_6":"class_99"].div(z2_new["sum"], axis=0)
z2_new.index = z2_new.index.map(int)

zf = (z_new + z2_new) / 2

zf.to_csv('single_combined_predictions_6.csv', index=True)

In [ ]:
z = pd.read_csv('single_combined_predictions.csv')

z = z.groupby('object_id').mean()

z2 = pd.read_csv('single_combined_predictions_2.csv')

z2 = z2.groupby('object_id').mean()

z3 = pd.read_csv('single_combined_predictions_3.csv')

z3 = z3.groupby('object_id').mean()

z4 = pd.read_csv('single_combined_predictions_4.csv')

z4 = z4.groupby('object_id').mean()

zf = (z + z2 + z3 + z4) / 4

zf.index = zf.index.map(int)

zf.to_csv('single_combined_predictions_many.csv', index=True)

In [ ]:
z = pd.read_csv('subm_0.658553_2018-12-09-21-42.csv')

print(z.groupby('object_id').size().max())
print((z.groupby('object_id').size() > 1).sum())

z = z.groupby('object_id').mean()

z_new = z
z_new["sum"] = z_new.sum(axis=1)

z_new = z_new.loc[:,"class_6":"class_99"]#.div(z_new["sum"], axis=0)
z_new['class_99'] = 0
z_new.index = z_new.index.map(int)

z_new.to_csv('gbm_single_predictions.csv', index=True)

In [ ]:
feats = ['class_6', 'class_15', 'class_16', 'class_42', 'class_52', 'class_53',
         'class_62', 'class_64', 'class_65', 'class_67', 'class_88', 'class_90',
         'class_92', 'class_95']

z = pd.read_csv('subm_0.570568_2018-12-14-16-13.csv')

print(z.groupby('object_id').size().max())
print((z.groupby('object_id').size() > 1).sum())

z = z.groupby('object_id').mean()

z2 = pd.read_csv('predictions.csv')

print(z2.groupby('object_id').size().max())
print((z2.groupby('object_id').size() > 1).sum())

z2 = z2.groupby('object_id').mean()

z['class_99'] = 0
z2['class_99'] = 0

z_sum = z[feats].sum(axis=1)

z = z.loc[:,"class_6":"class_95"].div(z_sum, axis=0)
z.index = z.index.map(int)

z2_sum = z2[feats].sum(axis=1)

z2 = z2.loc[:,"class_6":"class_95"].div(z2_sum, axis=0)
z2.index = z2.index.map(int)

zf = (z + z2) / 2

def GenUnknown(data):
    return ((((((data["mymedian"]) + (((data["mymean"]) / 2.0)))/2.0)) + (((((1.0) - (((data["mymax"]) * (((data["mymax"]) * (data["mymax"]))))))) / 2.0)))/2.0)

#def GenUnknown(data):
#    return (0.5 + 0.5 * data["mymedian"] + 0.25 * data["mymean"] - 0.5 * data["mymax"] ** 3) / 2

y = zf
y['mymean'] = zf[feats].mean(axis=1)
y['mymedian'] = zf[feats].median(axis=1)
y['mymax'] = zf[feats].max(axis=1)

zf['class_99'] = GenUnknown(y)

del zf['mymean'], zf['mymedian'], zf['mymax']

zf_new = zf
zf_new_sum = zf_new.sum(axis=1)

zf_new = zf_new.loc[:,"class_6":"class_99"].div(zf_new_sum, axis=0)
zf_new.index = zf_new.index.map(int)

zf_new.to_csv('single_combined_predictions_sn2.csv', index=True)

In [ ]:
zf_new.head()

In [11]:
feats = ['class_6', 'class_15', 'class_16', 'class_42', 'class_52', 'class_53',
         'class_62', 'class_64', 'class_65', 'class_67', 'class_88', 'class_90',
         'class_92', 'class_95']

z = pd.read_csv('single_combined_predictions_sn6.csv')

print(z.groupby('object_id').size().max())
print((z.groupby('object_id').size() > 1).sum())

z = z.groupby('object_id').mean()

def GenUnknown(data):
    return ((((((data["mymedian"]) + (((data["mymean"]) / 2.0)))/2.0)) + (((((1.0) - (((data["mymax"]) * (((data["mymax"]) * (data["mymax"]))))))) / 2.0)))/2.0)

#def GenUnknown(data):
#    return (0.5 + 0.5 * data["mymedian"] + 0.25 * data["mymean"] - 0.5 * data["mymax"] ** 3) / 2

y = z
y['mymean'] = z[feats].mean(axis=1)
y['mymedian'] = z[feats].median(axis=1)
y['mymax'] = z[feats].max(axis=1)

z['class_99'] = GenUnknown(y)

del z['mymean'], z['mymedian'], z['mymax']

z_new = z
z_new["sum"] = z_new.sum(axis=1)

z_new = z_new.loc[:,"class_6":"class_99"].div(z_new["sum"], axis=0)
z_new.index = z_new.index.map(int)

z_new.to_csv('single_combined_predictions_sn6_v3.csv', index=True)

1
0


In [2]:
feats = ['class_6', 'class_15', 'class_16', 'class_42', 'class_52', 'class_53',
         'class_62', 'class_64', 'class_65', 'class_67', 'class_88', 'class_90',
         'class_92', 'class_95']

z = pd.read_csv('subm_0.558025_2018-12-15-13-26.csv')

print(z.groupby('object_id').size().max())
print((z.groupby('object_id').size() > 1).sum())

z = z.groupby('object_id').mean()

z2 = pd.read_csv('predictions.csv')

print(z2.groupby('object_id').size().max())
print((z2.groupby('object_id').size() > 1).sum())

z2 = z2.groupby('object_id').mean()

z3 = pd.read_csv('subm_0.558025_2018-12-15-13-26.csv')

print(z.groupby('object_id').size().max())
print((z.groupby('object_id').size() > 1).sum())

z3 = z.groupby('object_id').mean()

z['class_99'] = 0
z2['class_99'] = 0
z3['class_99'] = 0

z_sum = z[feats].sum(axis=1)

z = z.loc[:,"class_6":"class_95"].div(z_sum, axis=0)
z.index = z.index.map(int)

z2_sum = z2[feats].sum(axis=1)

z2 = z2.loc[:,"class_6":"class_95"].div(z2_sum, axis=0)
z2.index = z2.index.map(int)

z3_sum = z2[feats].sum(axis=1)

z3 = z3.loc[:,"class_6":"class_95"].div(z3_sum, axis=0)
z3.index = z3.index.map(int)

zf = (z + z2 + z3) / 3

def GenUnknown(data):
    return ((((((data["mymedian"]) + (((data["mymean"]) / 2.0)))/2.0)) + (((((1.0) - (((data["mymax"]) * (((data["mymax"]) * (data["mymax"]))))))) / 2.0)))/2.0)

#def GenUnknown(data):
#    return (0.5 + 0.5 * data["mymedian"] + 0.25 * data["mymean"] - 0.5 * data["mymax"] ** 3) / 2

y = zf
y['mymean'] = zf[feats].mean(axis=1)
y['mymedian'] = zf[feats].median(axis=1)
y['mymax'] = zf[feats].max(axis=1)

zf['class_99'] = GenUnknown(y)

del zf['mymean'], zf['mymedian'], zf['mymax']

zf_new = zf
zf_new_sum = zf_new.sum(axis=1)

zf_new = zf_new.loc[:,"class_6":"class_99"].div(zf_new_sum, axis=0)
zf_new.index = zf_new.index.map(int)

zf_new.to_csv('single_combined_predictions_sn4.csv', index=True)

2
2
1
0
1
0


In [7]:
feats = ['class_6', 'class_15', 'class_16', 'class_42', 'class_52', 'class_53',
         'class_62', 'class_64', 'class_65', 'class_67', 'class_88', 'class_90',
         'class_92', 'class_95']

z = pd.read_csv('subm_0.528338_2018-12-15-15-11.csv')

print(z.groupby('object_id').size().max())
print((z.groupby('object_id').size() > 1).sum())

z = z.groupby('object_id').mean()

z2 = pd.read_csv('predictions_newmodel3.csv')

print(z2.groupby('object_id').size().max())
print((z2.groupby('object_id').size() > 1).sum())

z2 = z2.groupby('object_id').mean()

z3 = pd.read_csv('subm_0.528338_2018-12-15-15-11.csv')

print(z.groupby('object_id').size().max())
print((z.groupby('object_id').size() > 1).sum())

z3 = z.groupby('object_id').mean()

z['class_99'] = 0
z2['class_99'] = 0
z3['class_99'] = 0

z_sum = z[feats].sum(axis=1)

z = z.loc[:,"class_6":"class_95"].div(z_sum, axis=0)
z.index = z.index.map(int)

z2_sum = z2[feats].sum(axis=1)

z2 = z2.loc[:,"class_6":"class_95"].div(z2_sum, axis=0)
z2.index = z2.index.map(int)

z3_sum = z2[feats].sum(axis=1)

z3 = z3.loc[:,"class_6":"class_95"].div(z3_sum, axis=0)
z3.index = z3.index.map(int)

zf = (2.5*z + z2 + 2.5*z3) / 6

def GenUnknown(data):
    return ((((((data["mymedian"]) + (((data["mymean"]) / 2.0)))/2.0)) + (((((1.0) - (((data["mymax"]) * (((data["mymax"]) * (data["mymax"]))))))) / 2.0)))/2.0)

#def GenUnknown(data):
#    return (0.5 + 0.5 * data["mymedian"] + 0.25 * data["mymean"] - 0.5 * data["mymax"] ** 3) / 2

y = zf
y['mymean'] = zf[feats].mean(axis=1)
y['mymedian'] = zf[feats].median(axis=1)
y['mymax'] = zf[feats].max(axis=1)

zf['class_99'] = GenUnknown(y)

del zf['mymean'], zf['mymedian'], zf['mymax']

zf_new = zf
zf_new_sum = zf_new.sum(axis=1)

zf_new = zf_new.loc[:,"class_6":"class_99"].div(zf_new_sum, axis=0)
zf_new.index = zf_new.index.map(int)

zf_new.to_csv('single_combined_predictions_sn19.csv', index=True)

2
2
1
0
1
0


In [2]:
feats = ['class_6', 'class_15', 'class_16', 'class_42', 'class_52', 'class_53',
         'class_62', 'class_64', 'class_65', 'class_67', 'class_88', 'class_90',
         'class_92', 'class_95']

z = pd.read_csv('subm_0.521807_2018-12-16-18-39.csv')

print(z.groupby('object_id').size().max())
print((z.groupby('object_id').size() > 1).sum())

z = z.groupby('object_id').mean()

z2 = pd.read_csv('predictions_newmodel.csv')

print(z2.groupby('object_id').size().max())
print((z2.groupby('object_id').size() > 1).sum())

z2 = z2.groupby('object_id').mean()

z['class_99'] = 0
z2['class_99'] = 0

z_sum = z[feats].sum(axis=1)

z = z.loc[:,"class_6":"class_95"].div(z_sum, axis=0)
z.index = z.index.map(int)

z2_sum = z2[feats].sum(axis=1)

z2 = z2.loc[:,"class_6":"class_95"].div(z2_sum, axis=0)
z2.index = z2.index.map(int)

zf = (5*z + z2) / 6

def GenUnknown(data):
    return ((((((data["mymedian"]) + (((data["mymean"]) / 2.0)))/2.0)) + (((((1.0) - (((data["mymax"]) * (((data["mymax"]) * (data["mymax"]))))))) / 2.0)))/2.0)

#def GenUnknown(data):
#    return (0.5 + 0.5 * data["mymedian"] + 0.25 * data["mymean"] - 0.5 * data["mymax"] ** 3) / 2

y = zf
y['mymean'] = zf[feats].mean(axis=1)
y['mymedian'] = zf[feats].median(axis=1)
y['mymax'] = zf[feats].max(axis=1)

zf['class_99'] = GenUnknown(y)

del zf['mymean'], zf['mymedian'], zf['mymax']

zf_new = zf
zf_new_sum = zf_new.sum(axis=1)

zf_new = zf_new.loc[:,"class_6":"class_99"].div(zf_new_sum, axis=0)
zf_new.index = zf_new.index.map(int)

zf_new.to_csv('single_combined_predictions_sn14.csv', index=True)

2
2
1
0
